Unfortunately, it was not possible to directly query the API to retrieve the relevant information because the information that can be retrieved is too limited. For example, it was not possible to retrieve all papers that include topics of medicine and AI which is what this entire project is based on. 
Step 2 is done because combining all the csv files into one file would create a file of more than 20GB which is too large to be processed on my computer at once and lead to kernel crashes. 

# Preprocesing

In [ ]:
import requests
import pandas as pd

url = f"https://api.openalex.org/works?filter=from_publication_date:2019-01-01&per_page=200&cursor=*"
response = requests.get(url)

## Preprocessing and saving

Reading in all the files with pandas is not possible because all the csv files need together 20GB worth of storage which is too large to be read into memory all at once. Therefore, the dask library is used to preprocess the data. 

In [36]:
import dask.dataframe as dd
# files have been stored in a folder on an external hard drive
df = dd.read_csv("/Volumes/Festplatte/dataviz/data/openalex_*.csv") 
df.columns

Index(['title', 'id', 'doi', 'publication_date', 'language', 'type',
       'open_access', 'authors_name', 'authors_orcid', 'authors_id',
       'authors_positions', 'institution_names', 'institution_ids',
       'institution_types', 'institudion_countries', 'countries', 'citations',
       'topics', 'domains', 'fields', 'subfields', 'keywords', 'concepts',
       'referenced_work_count', 'counts_by_years', 'references'],
      dtype='object')

In [37]:
# safe main info for articles
df[df["type"] == "article"][
    ["title", "id", "doi", "publication_date", "language", "type", "open_access", "citations", "referenced_work_count"]
].to_csv("openalex_articles_main.csv", index=False, single_file=True)

# save the authors data
df[df["type"] == "article"][
    [
        "doi",
        "authors_name",
        "authors_orcid",
        "authors_id",
        "authors_positions",
        "institution_names",
        "institution_ids",
        "institution_types",
        "institudion_countries",
        "countries",
    ]
].to_csv("openalex_articles_authors_institutions.csv", index=False, single_file=True)

# save the topics data
df[df["type"] == "article"][["doi", "topics"]].to_csv("openalex_articles_topics.csv", index=False, single_file=True)

# save the domain data
df[df["type"] == "article"][["doi", "domains"]].to_csv("openalex_articles_domains.csv", index=False, single_file=True)

# save the fields data
df[df["type"] == "article"][["doi", "fields"]].to_csv("openalex_articles_fields.csv", index=False, single_file=True)

# save the subfields data
df[df["type"] == "article"][["doi", "subfields"]].to_csv(
    "openalex_articles_subfields.csv", index=False, single_file=True
)

# save concepts data
df[df["type"] == "article"][["doi", "concepts"]].to_csv("openalex_articles_concepts.csv", index=False, single_file=True)

# save authors data
df[df["type"] == "article"][
    [
        "doi",
        "authors_id",
        "authors_positions",
        "institution_ids",
        "institution_types",
        "institudion_countries",
        "countries",
    ]
].to_csv("openalex_articles_authors_institutions.csv", index=False, single_file=True)

# save the counts by years data
df[df["type"] == "article"][["doi", "counts_by_years"]].to_csv(
    "openalex_articles_citation_counts_by_year.csv", index=False, single_file=True
)

['/Users/lisa/Documents/DataScienceMaster/visualization/project/openalex_articles_main.csv']

In [38]:
# save the keywords data
df[df["type"] == "article"][["doi", "keywords"]].to_csv("openalex_articles_keywords.csv", index=False, single_file=True)

['/Users/lisa/Documents/DataScienceMaster/visualization/project/openalex_articles_keywords.csv']

In [6]:
import sqlite3
import time

path_to_csv = "/Volumes/Festplatte/dataviz/"
db_path = "openalex.db"

# Connect to the database
conn = sqlite3.connect(db_path)


def write_to_sql_with_retry(df, table_name, conn, chunksize=1000, max_retries=3):
    for attempt in range(max_retries):
        try:
            df.to_sql(name=table_name, con=conn, if_exists="replace", chunksize=chunksize)
            print("Data written to SQL successfully.")
            break
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt < max_retries - 1:
                time.sleep(5)  # Wait for 5 seconds before retrying
            else:
                print("Max retries reached. Failed to write data to SQL.")
                raise

In [7]:
df = pd.read_csv(f"{path_to_csv}openalex_articles_main.csv")
write_to_sql_with_retry(df, "main", conn)

Data written to SQL successfully.


In [8]:
df = pd.read_csv(f"{path_to_csv}openalex_articles_keywords.csv")
df["num_keywords"] = df["keywords"].apply(lambda x: len(x.split(",")))
df.keywords = df.keywords.transform(lambda x: x[2:-2].split("', '"))
df = df.explode("keywords")
write_to_sql_with_retry(df, "keywords", conn)

Data written to SQL successfully.


In [1]:
df = pd.read_csv("openalex_articles_authors_institutions.csv")

In [2]:
df.head()

,doi,authors_id,authors_positions,institution_ids,institution_types,institudion_countries,countries
0,https://doi.org/10.3322/caac.21660,"['https://openalex.org/A5082191284', 'https://...","['first', 'middle', 'middle', 'middle', 'middl...","['https://openalex.org/I1288198617', 'https://...","['nonprofit', 'government', 'nonprofit', 'gove...","['US', 'FR', 'US', 'FR', 'FR', 'US', 'FR']","[['US'], ['FR'], ['US'], ['FR'], ['FR'], ['US'..."
1,https://doi.org/10.1016/s0140-6736(20)30183-5,"['https://openalex.org/A5101545030', 'https://...","['first', 'middle', 'middle', 'middle', 'middl...","['https://openalex.org/I200296433', 'https://o...","['education', 'healthcare', 'education', 'heal...","['CN', 'CN', 'CN', 'CN', 'CN', 'CN', 'CN', 'CN...","[['CN'], ['CN'], ['CN'], ['CN'], ['CN'], ['CN'..."
2,https://doi.org/10.1136/bmj.n71,"['https://openalex.org/A5089979303', 'https://...","['first', 'middle', 'middle', 'middle', 'middl...","['https://openalex.org/I56590836', 'https://op...","['education', 'education', 'education', 'gover...","['AU', 'AU', 'NL', 'FR', 'FR', None, 'AU', 'SG...","[['AU'], ['AU'], ['NL'], ['FR'], ['AU'], ['SG'..."
3,https://doi.org/10.18653/v1/n19-1423,"['https://openalex.org/A5057457287', 'https://...","['first', 'middle', 'middle', 'last']",[],[],[],"[[], [], [], []]"
4,https://doi.org/10.1056/nejmoa2002032,"['https://openalex.org/A5101633572', 'https://...","['first', 'middle', 'middle', 'middle', 'middl...","['https://openalex.org/I58200834', 'https://op...","['education', 'education', 'healthcare', 'heal...","['CN', 'CN', 'CN', 'CN', 'CN', 'CN', 'CN', 'CN...","[['CN'], ['CN'], ['CN'], ['CN'], ['CN'], ['CN'..."


## Save csv files in Database

In [9]:
df = pd.read_csv(f"{path_to_csv}openalex_articles_authors_institutions.csv")
df["num_authors"] = df["authors_id"].apply(lambda x: len(x.split(",")))

df["num_countries"] = df["countries"].apply(lambda x: len(x.split(",")))

df.authors_id = df.authors_id.transform(lambda x: x[2:-2].split("', '"))
df.authors_positions = df.authors_positions.transform(lambda x: x[2:-2].split("', '"))

df.countries = df.countries.transform(lambda x: x[2:-2].split("', '"))

df = df[
    [
        "doi",
        "authors_id",
        "authors_positions",
        "num_authors",
    ]
].explode(["authors_id", "authors_positions"])

In [10]:
write_to_sql_with_retry(df, "authors", conn)

Data written to SQL successfully.


In [11]:
# safe institutions data
df = pd.read_csv(f"{path_to_csv}openalex_articles_authors_institutions.csv")
df.institution_ids = df.institution_ids.transform(lambda x: x[2:-2].split("', '"))
df.institution_types = df.institution_types.transform(lambda x: x[2:-2].split("', '"))
df.institudion_countries = df.institudion_countries.transform(lambda x: x[2:-2].split("', '"))
# only include rows where the number of institutions is equal to the number of institution types and countries
df["num_institutions"] = df["institution_ids"].apply(lambda x: len(x))
df["num_institution_types"] = df["institution_types"].apply(lambda x: len(x))
df["num_institudion_countries"] = df["institudion_countries"].apply(lambda x: len(x))
df = df[df["num_institutions"] == df["num_institution_types"]]
df = df[df["num_institutions"] == df["num_institudion_countries"]]

df = df[["doi", "institution_ids", "institution_types", "institudion_countries", "num_institutions"]].explode(
    ["institution_ids", "institution_types", "institudion_countries"]
)

# safe institutions data
write_to_sql_with_retry(df, "institutions", conn)

Data written to SQL successfully.


In [12]:
df = pd.read_csv(f"{path_to_csv}openalex_articles_authors_institutions.csv")
df["num_countries"] = df["countries"].apply(lambda x: len(x.split(",")))
df.countries = df.countries.transform(lambda x: x[2:-2].split("', '"))
df = df[["doi", "countries", "num_countries"]].explode("countries")
write_to_sql_with_retry(df, "countries", conn)

Data written to SQL successfully.


In [13]:
df = pd.read_csv(f"{path_to_csv}openalex_articles_concepts.csv")
df.concepts = df.concepts.transform(lambda x: x[2:-2].split("', '"))
df["num_concepts"] = df["concepts"].apply(lambda x: len(x))
df = df.explode("concepts")
write_to_sql_with_retry(df, "concepts", conn)

Data written to SQL successfully.


In [14]:
df = pd.read_csv(f"{path_to_csv}openalex_articles_domains.csv")
df.domains = df.domains.transform(lambda x: x[2:-2].split("', '"))
df = df.explode("domains")
write_to_sql_with_retry(df, "domains", conn)

Data written to SQL successfully.


In [15]:
df = pd.read_csv(f"{path_to_csv}openalex_articles_fields.csv")
df.fields = df.fields.transform(lambda x: x[2:-2].split("', '"))
df = df.explode("fields")
write_to_sql_with_retry(df, "fields", conn)

Data written to SQL successfully.


In [16]:
df = pd.read_csv(f"{path_to_csv}openalex_articles_subfields.csv")
df.subfields = df.subfields.transform(lambda x: x[2:-2].split("', '"))
df = df.explode("subfields")
write_to_sql_with_retry(df, "subfields", conn)

Data written to SQL successfully.


In [17]:
df = pd.read_csv(f"{path_to_csv}openalex_articles_topics.csv")
df.topics = df.topics.transform(lambda x: x[2:-2].split("', '"))
df = df.explode("topics")
write_to_sql_with_retry(df, "topics", conn)

Data written to SQL successfully.


In [18]:
df = pd.read_csv(f"{path_to_csv}openalex_articles_citation_counts_by_year.csv")

In [19]:
df.head()

,doi,counts_by_years
0,https://doi.org/10.3322/caac.21660,"[{'year': 2024, 'cited_by_count': 21589}, {'ye..."
1,https://doi.org/10.1016/s0140-6736(20)30183-5,"[{'year': 2024, 'cited_by_count': 2113}, {'yea..."
2,https://doi.org/10.1136/bmj.n71,"[{'year': 2024, 'cited_by_count': 13280}, {'ye..."
3,https://doi.org/10.18653/v1/n19-1423,"[{'year': 2024, 'cited_by_count': 799}, {'year..."
4,https://doi.org/10.1056/nejmoa2002032,"[{'year': 2024, 'cited_by_count': 1100}, {'yea..."


In [20]:
import json
import re
import ast

values = []
for doi, article in zip(df.doi, df["counts_by_years"]):
    row = []
    article = article.replace("'", '"')
    article = ast.literal_eval(article)
    row.append(doi)
    for year in range(2009, 2024):
        found_year = False
        for x in article:
            if x["year"] == year:
                found_year = True
                row.append(x["cited_by_count"])
        if not found_year:
            row.append(0)
    values.append(row)

df = pd.DataFrame(
    values,
    columns=[
        "doi",
        "2010",
        "2011",
        "2012",
        "2013",
        "2014",
        "2015",
        "2016",
        "2017",
        "2018",
        "2019",
        "2020",
        "2021",
        "2022",
        "2023",
        "2024",
    ],
)

df.head()

,doi,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,https://doi.org/10.3322/caac.21660,0,0,0,0,0,1,0,1,0,6,11,24,6055,20928,26485
1,https://doi.org/10.1016/s0140-6736(20)30183-5,0,0,0,0,1,0,3,1,2,4,37,19047,13601,8034,4829
2,https://doi.org/10.1136/bmj.n71,0,0,0,0,0,0,0,0,0,0,0,9,2470,9447,13925
3,https://doi.org/10.18653/v1/n19-1423,0,0,0,0,1,0,0,0,6,85,3475,9007,11548,2753,2769
4,https://doi.org/10.1056/nejmoa2002032,0,0,0,2,1,0,2,0,0,1,28,12234,8245,4494,2515


In [21]:
write_to_sql_with_retry(df, "citations", conn)

Data written to SQL successfully.


# Data Analysis

In [4]:
import sqlite3

db_path = "openalex.db"

# Connect to the database
conn = sqlite3.connect(db_path)

cursor = conn.cursor()
# Get the table names and column names from the database
cursor.execute(
    """
    SELECT name FROM sqlite_master WHERE type='table';
    """
)
cursor.fetchall()

[('main',),
 ('keywords',),
 ('authors',),
 ('institutions',),
 ('countries',),
 ('concepts',),
 ('domains',),
 ('fields',),
 ('subfields',),
 ('topics',),
 ('citations',)]

In [2]:
cursor = conn.cursor()
cursor.execute(
    """
    SELECT DISTINCT fields FROM fields;
    """
)
cursor.fetchall()

[('Medicine',),
 ('Decision Sciences',),
 ('Social Sciences',),
 ('Computer Science',),
 ('Biochemistry, Genetics and Molecular Biology',),
 ('Materials Science',),
 ('Agricultural and Biological Sciences',),
 ('Economics, Econometrics and Finance',),
 ('Environmental Science',),
 ('Earth and Planetary Sciences',),
 ('Mathematics',),
 ('Psychology',),
 ('',),
 ('Health Professions',),
 ('Physics and Astronomy',),
 ('Immunology and Microbiology',),
 ('Engineering',),
 ('Chemistry',),
 ('Arts and Humanities',),
 ('Business, Management and Accounting',),
 ('Pharmacology, Toxicology and Pharmaceutics',),
 ('Neuroscience',),
 ('Energy',),
 ('Chemical Engineering',),
 ('Dentistry',),
 ('Nursing',),
 ('Veterinary',)]

In [3]:
cursor.execute(
    """
    SELECT DISTINCT subfields FROM subfields;
    """
)
cursor.fetchall()

[('Oncology',),
 ('Epidemiology',),
 ('Infectious Diseases',),
 ('Statistics, Probability and Uncertainty',),
 ('Sociology and Political Science',),
 ('Artificial Intelligence',),
 ('General Social Sciences',),
 ('Neurology',),
 ('Molecular Biology',),
 ('Materials Chemistry',),
 ('Animal Science and Zoology',),
 ('Radiology, Nuclear Medicine and Imaging',),
 ('Economics and Econometrics',),
 ('Global and Planetary Change',),
 ('Atmospheric Science',),
 ('Cancer Research',),
 ('Modeling and Simulation',),
 ('Clinical Psychology',),
 ('',),
 ('Computer Vision and Pattern Recognition',),
 ('Information Systems and Management',),
 ('Information Systems',),
 ('Management Science and Operations Research',),
 ('Cardiology and Cardiovascular Medicine',),
 ('Genetics',),
 ('Biophysics',),
 ('Health, Toxicology and Mutagenesis',),
 ('Health',),
 ('General Health Professions',),
 ('Atomic and Molecular Physics, and Optics',),
 ('Pulmonary and Respiratory Medicine',),
 ('Experimental and Cognitiv

In [4]:
query = """
    SELECT 
    keywords.doi,
    keywords, 
    [2019], [2020], [2021], [2022], [2023], [2024]
    FROM 
        keywords
    LEFT JOIN 
        citations 
        ON keywords.doi = citations.doi
    """

df_keywords = pd.read_sql_query(query, conn)
df_keywords.head()

,doi,keywords,2019,2020,2021,2022,2023,2024
0,https://doi.org/10.3322/caac.21660,Cancer Incidence,6.0,11.0,24.0,6055.0,20928.0,26485.0
1,https://doi.org/10.3322/caac.21660,Cancer Registry Data,6.0,11.0,24.0,6055.0,20928.0,26485.0
2,https://doi.org/10.3322/caac.21660,Stomach cancer,6.0,11.0,24.0,6055.0,20928.0,26485.0
3,https://doi.org/10.1016/s0140-6736(20)30183-5,myalgia,4.0,37.0,19047.0,13601.0,8034.0,4829.0
4,https://doi.org/10.1016/s0140-6736(20)30183-5,Corona Virus,4.0,37.0,19047.0,13601.0,8034.0,4829.0


## AI in Medicine
**Which papers are considered to be at the intersection of AI in medicine?**

Each paper has three domains, three fields, three subfields, and three topics in this hierarchical order. Papers are considered to be in the intersection of AI in medicine if they have a field in medicine and a subfield in either 'Artificial Intelligence' or 'Computer Vision and Pattern Recognition', which in this case is also considered to be AI. In this way, not all papers with topics in both fields will be considered, but at least the vast majority should be.

Blue to purple colors are chosen for relations to medicine. Therefore, the other areas have a grey color, but for all considerations related to medicine, the blue to purple colors are chosen. This choice also takes into account different forms of color blindness.

In [2]:
color_medicine = "#4063FF"
color_other_fields = "grey"

image_color = ["#9940FF"]
nlp_color = ["#5A40FF"]
explainable_color = ["#409FFF"]
color_rest = ["#CBD2FF"]

top_color = "#29D7FF"
bottom_color = "#BA2EDF"

### Plot 1

In [5]:
# get subfields from the subfield of AI and Computer Vision
query = """
    SELECT subfields, fields, topics, main.doi, publication_date, COUNT(main.doi) as count
    FROM 
        subfields
    LEFT JOIN
        fields
        ON subfields.doi = fields.doi 
    LEFT JOIN
        topics
        ON subfields.doi = topics.doi
    LEFT JOIN
        main
        ON subfields.doi = main.doi
    WHERE 
        subfields = 'Artificial Intelligence' OR subfields = 'Computer Vision and Pattern Recognition'
    GROUP BY subfields, fields, topics, main.doi, publication_date;
    """
df = pd.read_sql_query(query, conn)

In [6]:
# get all the papers with subfield AI and Computer Vision
query = """
    SELECT subfields, publication_date, COUNT(main.doi) as count
    FROM 
        subfields
    LEFT JOIN
        main
        ON subfields.doi = main.doi
    WHERE 
        subfields = 'Artificial Intelligence' OR subfields = 'Computer Vision and Pattern Recognition'
    GROUP BY subfields, main.doi, publication_date;
    """
df_ai = pd.read_sql_query(query, conn)

df_ai.dropna(inplace=True)
df_ai["year"] = df_ai["publication_date"].apply(lambda x: x[:4])
df_ai = df_ai.groupby("year").sum().reset_index()

In [43]:
# create subplots
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df_med = df.copy()[df["fields"] == "Medicine"]
df_other = df.copy()[df["fields"] != "Medicine"]
df_med["year"] = df_med["publication_date"].apply(lambda x: int(x[:4]))
df_med = df_med[df_med["year"] != 2024]  # 2024 not complete
df_med = df_med.groupby("year").size().reset_index()
df_med.columns = ["year", "count"]

df_other.dropna(inplace=True)
num_fields = df_other["fields"].nunique()
df_other["year"] = df_other["publication_date"].apply(lambda x: x[:4])
df_other = df_other[df_other["year"] != "2024"]  # 2024 is not complete
df_other = df_other.groupby("year").size().reset_index()
df_other.columns = ["year", "count"]
df_other["count"] = df_other["count"] / num_fields

fig = make_subplots(rows=1, cols=1)
fig.add_trace(
    go.Scatter(
        x=df_med["year"], y=df_med["count"], mode="lines+markers", name="Medicine", line=dict(color=color_medicine)
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=df_other["year"],
        y=df_other["count"],
        mode="lines+markers",
        name="Other Fields",
        line=dict(color=color_other_fields),
    ),
    row=1,
    col=1,
)
# remove background and gridlines
fig.update_layout(
    paper_bgcolor="white",
    plot_bgcolor="white",
    yaxis=dict(
        showgrid=True,
        gridcolor="lightgray",
        range=[0, 16000],
    ),
)
fig.add_annotation(
    x=2023,
    y=15300,
    text=" Medicine",
    ax=0,
    ay=0,
    xanchor="left",
    font=dict(size=16, color=color_medicine, weight="bold"),
)

fig.add_annotation(
    x=2023,
    y=9300,
    text=" Other Fields",
    ax=0,
    ay=0,
    xanchor="left",
    font=dict(size=16, color=color_other_fields, weight="bold"),
)

fig.add_annotation(
    xref="paper",
    yref="paper",
    x=0,
    y=-0.15,
    text="The number of papers published in association with AI shows a <b>larger increase</b> compared to the average of the other fields. <br> In medicine, the number of papers has <b>almost doubled</b> since 2019.",
    ax=0,
    ay=0,
    xanchor="left",
    yanchor="top",
)

fig.add_annotation(
    x=2019,
    y=df_med[df_med["year"] == 2019]["count"].values[0],
    text=str(df_med[df_med["year"] == 2019]["count"].values[0]),
    ax=0,
    ay=20,
    xanchor="center",
    font=dict(size=12, color=color_medicine, weight="bold"),
)

fig.add_annotation(
    x=2023,
    y=df_med[df_med["year"] == 2023]["count"].values[0],
    text=str(df_med[df_med["year"] == 2023]["count"].values[0]),
    ax=-40,
    ay=-10,
    xanchor="center",
    font=dict(size=12, color=color_medicine, weight="bold"),
)

# add title
fig.update_layout(
    title_text=f'The Number of Papers Published With <B><span style="color: {color_medicine};">AI in Medicine</span></B> Is Increasing <B>Stronger</B> Than in <span style="color: {color_other_fields};">Other Fields</span>',
    xaxis_title="",
    yaxis_title="Number of Papers",
    xaxis=dict(
        tickmode="linear",
        tick0=2019,
        dtick=1,
    ),
    width=1000,
    height=600,
    margin=dict(b=120),
)
# set template
fig.update_layout(
    template="simple_white",
    showlegend=False,
    margin=dict(
        l=110,
    ),
)
fig.write_image("relevance_plot.png", scale=10)
fig.write_image("relevance_plot.svg", scale=10)
fig.show()

The plot uses a simple_white template with only very thin and light horizontal grid lines to avoid clutter. When trying different options, it seemed that the grid lines were very helpful to better see the upward trend because otherwise the other fields line is used as a reference. The medicine line is still increasing more than the other fields line, but using the horizontal lines as a reference makes it much more visible. 
The year 2024 is not considered in this visualization because at the time of download (November 2024) the year was not yet complete, which would have distorted the graph. 

**Assumptions:** The papers that have medical and artificial intelligence concepts. Have topics in both, medicine and AI. If this is not the case, then they are not included in this. 

### Plot 2

In [41]:
# get trending topics in medicine with ai
import pandas as pd

df_med = df.copy()[df["fields"] == "Medicine"]
df_med["year"] = df_med["publication_date"].apply(lambda x: x[:4])

df_meds = df_med.groupby(["topics"]).count().reset_index().sort_values(by="doi", ascending=True).tail(20)

# color label words
edited_labels = []
for label in df_meds["topics"]:
    if "Images" in label:
        edited_labels.append(label.replace("Images", f'<span style="color: {image_color[0]};"><b>Images</b></span>'))
    elif "Image" in label:
        edited_labels.append(label.replace("Image", f'<span style="color: {image_color[0]};"><b>Image</b></span>'))
    elif "Imaging" in label:
        edited_labels.append(label.replace("Imaging", f'<span style="color: {image_color[0]};"><b>Imaging</b></span>'))
    elif "Natural Language Processing" in label:
        edited_labels.append(
            label.replace(
                "Natural Language Processing",
                f'<span style="color: {nlp_color[0]};"><b>Natural Language Processing</b></span>',
            )
        )
    elif "Explainable" in label:
        edited_labels.append(
            label.replace("Explainable", f'<span style="color: {explainable_color[0]};"><b>Explainable</b></span>')
        )
    else:
        edited_labels.append(label)


fig = go.Figure(
    data=[
        go.Bar(x=df_meds["doi"], y=edited_labels, orientation="h"),
    ]
)
fig.update_layout(
    title=f'<b><span style="color: {image_color[0]};">Computer Vision</span></b>, <b><span style="color: {explainable_color[0]};">Explainable AI</span></b> and <b><span style="color: {nlp_color[0]};">NLP</span></b> - The Most Relevant Topics',
    width=800,
    height=1000,
    xaxis=dict(
        tickformat=",",
    ),
)

# color certain bars
num_bars = len(df_meds["doi"])

colors = (
    (color_rest * 2)
    + nlp_color
    + image_color
    + color_rest
    + explainable_color
    + (color_rest * 3)
    + image_color
    + (color_rest * 3)
    + image_color
    + color_rest
    + image_color
    + color_rest
    + (image_color * 3)
)
fig.update_traces(
    marker=dict(
        color=colors
        # line=dict(color='rgb(8,48,107)', width=1.5)
    ),
    opacity=1.0,
)
fig.add_annotation(
    xref="paper",
    yref="paper",
    x=0.8,
    y=0.1,
    text=f'The <b>top 20 topics</b> in terms of number of papers<br>published in AI in Medicine over the last 5 years<br>show that <b><span style="color: {image_color[0]};">Computer Vision</span></b>, <b><span style="color: {explainable_color[0]};">Explainable AI</span></b> and<br><b><span style="color: {nlp_color[0]};">NLP</span></b> are very relevant topics. ',
    ax=0,
    ay=0,
    align="left",
    font=dict(
        size=10,
        color="black",
    ),
)

# add annotations for the colors
fig.update_layout(template="simple_white", xaxis=dict(title="Number of Papers"))
fig.write_image("topics_plot.png", scale=10)
fig.write_image("topics_plot.svg", scale=10)
fig.show()

Grid lines and other clutter are removed and attention is drawn to stronger colors, color highlights, and bold text. The caption is used to better explain the plot and highlight the main takeaway, which is also stated in the heading. A vertical bar graph is used to avoid rotated labels. 

### Plot 3

In [9]:
# get all the subfields of the field medicine form openalex api
response = requests.get(
    "https://api.openalex.org/works?group_by=primary_topic.subfield.id&per_page=200&filter=primary_topic.field.id:fields/27"
)
response.json()

# get list of subfields
subfields_medicine = []
for i in range(len(response.json()["group_by"])):
    subfields_medicine.append(response.json()["group_by"][i]["key_display_name"])

In [10]:
# get all the subfields of the field computer science form openalex api
response = requests.get(
    "https://api.openalex.org/works?group_by=primary_topic.subfield.id&per_page=200&filter=primary_topic.field.id:fields/17"
)
response.json()

# get list of subfields
subfields_computer_science = []
for i in range(len(response.json()["group_by"])):
    subfields_computer_science.append(response.json()["group_by"][i]["key_display_name"])

In [11]:
# get the paper subfields for the given dios
dios_med_ai = df_med["doi"].unique()
dios_med_ai = tuple(dios_med_ai)
query = f"""
    SELECT subfields, subfields.doi, publication_date
    FROM
        subfields
    JOIN main
    ON subfields.doi = main.doi
    WHERE 
        subfields.doi IN {dios_med_ai};
    """

subfields_medicine_ai = pd.read_sql_query(query, conn)
subfields_medicine_ai.head()

,subfields,doi,publication_date
0,Artificial Intelligence,https://doi.org/10.1001/amajethics.2019.167,2019-02-01
1,General Health Professions,https://doi.org/10.1001/amajethics.2019.167,2019-02-01
2,Epidemiology,https://doi.org/10.1001/amajethics.2019.167,2019-02-01
3,Health Informatics,https://doi.org/10.1001/amajethics.2019.180,2019-02-01
4,"Public Health, Environmental and Occupational ...",https://doi.org/10.1001/amajethics.2019.180,2019-02-01


In [12]:
query = f"""
    SELECT subfields, topics, subfields.doi, publication_date
    FROM subfields
    JOIN topics
    ON subfields.doi = topics.doi
    JOIN main
    ON subfields.doi = main.doi
    WHERE subfields.subfields IN {tuple(subfields_medicine)};
    """

df_all_medicine = pd.read_sql_query(query, conn)

In [13]:
subfields_total = df_all_medicine.groupby("subfields").count()
subfields_total.head()

,topics,doi,publication_date
subfields,,,
Anatomy,782,782,782
Anesthesiology and Pain Medicine,17309,17309,17309
Biochemistry,97397,97397,97397
Cardiology and Cardiovascular Medicine,251568,251568,251568
Complementary and alternative medicine,56134,56134,56134


In [14]:
subfields_medicine_ai

,subfields,doi,publication_date
0,Artificial Intelligence,https://doi.org/10.1001/amajethics.2019.167,2019-02-01
1,General Health Professions,https://doi.org/10.1001/amajethics.2019.167,2019-02-01
2,Epidemiology,https://doi.org/10.1001/amajethics.2019.167,2019-02-01
3,Health Informatics,https://doi.org/10.1001/amajethics.2019.180,2019-02-01
4,"Public Health, Environmental and Occupational ...",https://doi.org/10.1001/amajethics.2019.180,2019-02-01
...,...,...,...
66634,Health Informatics,https://doi.org/10.9781/ijimai.2024.02.008,2024-01-01
66635,Artificial Intelligence,https://doi.org/10.9781/ijimai.2024.02.008,2024-01-01
66636,Safety Research,https://doi.org/10.9785/cri-2019-200402,2019-08-01
66637,Health Informatics,https://doi.org/10.9785/cri-2019-200402,2019-08-01


In [15]:
subfields_medicine_ai = subfields_medicine_ai[subfields_medicine_ai.subfields.isin(subfields_total.index)]
df_group = subfields_medicine_ai.copy().groupby("subfields").size().reset_index().sort_values(by=0, ascending=True)
df_group = pd.merge(df_group, subfields_total.reset_index(), on="subfields")
df_group["AI_percentage"] = (df_group[0] / df_group["doi"]) * 100
df_group = df_group[df_group["subfields"] != "Health Informatics"]
df_group_top = df_group.sort_values(by="AI_percentage", ascending=True).tail(11)
df_group_bottom = df_group.sort_values(by="AI_percentage", ascending=True).head(11)

df_group = df_group.sort_values(
    by="AI_percentage",
    ascending=True,
)

In [16]:
df_group_top

,subfields,0,topics,doi,publication_date,AI_percentage
5,Anatomy,3,782,782,782,0.383632
30,"Endocrinology, Diabetes and Metabolism",498,122717,122717,122717,0.405812
26,Complementary and alternative medicine,229,56134,56134,56134,0.407952
29,Psychiatry and Mental health,457,108574,108574,108574,0.420911
38,Epidemiology,2099,387163,387163,387163,0.542149
36,Neurology,1306,226894,226894,226894,0.575599
39,Oncology,2210,373993,373993,373993,0.590920
19,Otorhinolaryngology,100,15181,15181,15181,0.658718
34,Ophthalmology,718,64667,64667,64667,1.110304
25,Family Practice,171,9959,9959,9959,1.717040


In [36]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

opacities_top = [0.5] * 10 + [1.0]
opacities_bottom = [1.0] + [0.5] * 9 + [1.0]
fig.add_trace(
    go.Bar(
        x=df_group_top["AI_percentage"],
        y=df_group_top["subfields"],
        orientation="h",
        name="Top 10",
        marker=dict(color=top_color, opacity=opacities_top),
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Bar(
        x=df_group_bottom["AI_percentage"],
        y=df_group_bottom["subfields"],
        orientation="h",
        name="Bottom 10",
        marker=dict(color=bottom_color, opacity=opacities_bottom),
    ),
    row=2,
    col=1,
)

# Define y-axis labels with some transparency
y_labels_top = df_group_top["subfields"].tolist()
y_labels_bottom = df_group_bottom["subfields"].tolist()

# Make some labels slightly transparent by wrapping them in HTML with CSS
transparent_labels_top = [
    f'<span style="opacity:0.5">{label}</span>' if condition else label
    for label, condition in zip(y_labels_top, [True, True, True, True, True, True, True, True, True, True, False])
]
transparent_labels_bottom = [
    f'<span style="opacity:0.5">{label}</span>' if condition else label
    for label, condition in zip(y_labels_bottom, [False, True, True, True, True, True, True, True, True, True, False])
]

fig.update_yaxes(tickvals=df_group_top["subfields"], ticktext=transparent_labels_top, row=1, col=1)
fig.update_yaxes(tickvals=df_group_bottom["subfields"], ticktext=transparent_labels_bottom, row=2, col=1)

# define brackets for the top and bottom graph
shapes = [
    dict(
        type="line", x0=1.02, y0=1.0, x1=1.02, y1=0.55, xref="paper", yref="paper", line=dict(color=top_color, width=2)
    ),
    dict(
        type="line", x0=1.02, y0=1.0, x1=1.00, y1=1.0, xref="paper", yref="paper", line=dict(color=top_color, width=2)
    ),
    dict(
        type="line", x0=1.02, y0=0.55, x1=1.00, y1=0.55, xref="paper", yref="paper", line=dict(color=top_color, width=2)
    ),
    # bottom plot
    dict(
        type="line",
        x0=1.02,
        y0=-0.015,
        x1=1.02,
        y1=0.45,
        xref="paper",
        yref="paper",
        line=dict(color=bottom_color, width=2),
    ),
    dict(
        type="line",
        x0=1.02,
        y0=0.45,
        x1=1.00,
        y1=0.45,
        xref="paper",
        yref="paper",
        line=dict(color=bottom_color, width=2),
    ),
    dict(
        type="line",
        x0=1.02,
        y0=-0.015,
        x1=1.00,
        y1=-0.015,
        xref="paper",
        yref="paper",
        line=dict(color=bottom_color, width=2),
    ),
]

# define all the annotations
annotations = [
    dict(
        x=df_group_bottom.loc[df_group_bottom["subfields"] == "Pharmacology", "AI_percentage"].values[0],
        y="Pharmacology",
        text=f'{df_group_bottom.loc[df_group_bottom["subfields"] == "Pharmacology", "AI_percentage"].values[0]:.4f}%',
        xref="x2",
        yref="y2",
        showarrow=False,
        xanchor="left",
        font=dict(size=12, weight="bold", color=bottom_color),
    ),
    dict(
        x=df_group_bottom.loc[df_group_bottom["subfields"] == "Genetics", "AI_percentage"].values[0],
        y="Genetics",
        text=f'{df_group_bottom.loc[df_group_bottom["subfields"] == "Genetics", "AI_percentage"].values[0]:.2f}%',
        xref="x2",
        yref="y2",
        xanchor="left",
        showarrow=False,
        font=dict(size=12, weight="bold", color=bottom_color),
    ),
    dict(
        x=df_group_top[df_group_top["subfields"] == "Radiology, Nuclear Medicine and Imaging"]["AI_percentage"].values[
            0
        ]
        - 0.5,
        y="Radiology, Nuclear Medicine and Imaging",
        text=f'{df_group_top[df_group_top["subfields"] == "Radiology, Nuclear Medicine and Imaging"]["AI_percentage"].values[0]:.2f}%',
        xref="x1",
        yref="y1",
        ax=0,
        ay=0,
        # showarrow=False,
        font=dict(size=12, weight="bold", color="white"),
    ),
    dict(
        x=1.05,
        y=0.78,
        xref="paper",
        yref="paper",
        text="<b>largest</b> proportion",
        xanchor="left",
        showarrow=False,
        font=dict(size=12, color=top_color),
    ),
    dict(
        x=1.05,
        y=0.23,
        xanchor="left",
        xref="paper",
        yref="paper",
        text="<b>smallest</b> proportion",
        showarrow=False,
        font=dict(size=12, color=bottom_color),
    ),
    dict(
        xref="paper",
        yref="paper",
        x=1.0,
        y=0.1,
        text=f'The percentage of papers with AI in each<br>subfield of medicine <b>varies greatly between subfields</b>.<br>The <b><span style="color: {top_color};">top</span></b> graph shows the <b>11</b> subfields with the<br><b><span style="color: {top_color};">highest proportion</span></b>, while the <b><span style="color: {bottom_color};">bottom</span></b> graph shows<br>the <b>11</b> subfields with the <b><span style="color: {bottom_color};">lowest proportion</span></b>.',
        ax=0,
        ay=0,
        xanchor="right",
        align="left",
        font=dict(size=9),
    ),
]

fig.update_layout(
    width=800,
    height=800,
    xaxis=dict(range=[0, 5]),
    xaxis2=dict(title="Percentage of Papers with AI", range=[0, 6]),
    title_text="Percentage of Medicine Papers With AI <B>Greatly Varies</B> Between Subfields",
    annotations=annotations,
    shapes=shapes,
)
fig.update_layout(template="simple_white", showlegend=False, margin=dict(r=150))
fig.write_image("subfields_plot.png", scale=10)
fig.write_image("subfields_plot.svg", scale=10)
fig.show()

The largest and smallest subfields are clearly separated by space and color. The main takeaway is stated in the title and the main content is explained in the caption, both of which use bold text to emphasize important aspects. Grid lines and background is removed to avoid clutter. 
Annotations and highlighting are used for the relevant topics mentioned in the main text of the article.

In [18]:
df_all_medicine["year"] = [x[:4] for x in df_all_medicine["publication_date"]]
subfields_total = (
    df_all_medicine[df_all_medicine["year"] != "2024"].groupby(["subfields", "year"]).count().reset_index()
)
subfields_total.head(10)

,subfields,year,topics,doi,publication_date
0,Anatomy,2019,80,80,80
1,Anatomy,2020,82,82,82
2,Anatomy,2021,79,79,79
3,Anatomy,2022,57,57,57
4,Anatomy,2023,125,125,125
5,Anesthesiology and Pain Medicine,2019,3153,3153,3153
6,Anesthesiology and Pain Medicine,2020,2788,2788,2788
7,Anesthesiology and Pain Medicine,2021,2017,2017,2017
8,Anesthesiology and Pain Medicine,2022,1983,1983,1983
9,Anesthesiology and Pain Medicine,2023,2135,2135,2135


### Plot 4

In [20]:
query = f"""
    SELECT title, main.doi, publication_date, citations, topics
    FROM main
    LEFT JOIN topics
        ON main.doi = topics.doi
    WHERE main.doi IN {dios_med_ai};
    """

main_medicine_ai = pd.read_sql_query(query, conn)
main_medicine_ai.head()

,title,doi,publication_date,citations,topics
0,Can AI Help Reduce Disparities in General Medi...,https://doi.org/10.1001/amajethics.2019.167,2019-02-01,268,Deep Learning Applications in Healthcare
1,Can AI Help Reduce Disparities in General Medi...,https://doi.org/10.1001/amajethics.2019.167,2019-02-01,268,Strategies to Reduce Low-Value Health Care Ser...
2,Can AI Help Reduce Disparities in General Medi...,https://doi.org/10.1001/amajethics.2019.167,2019-02-01,268,Epidemiology and Implications of Multimorbidit...
3,What Are Important Ethical Implications of Usi...,https://doi.org/10.1001/amajethics.2019.180,2019-02-01,56,Artificial Intelligence in Medicine
4,What Are Important Ethical Implications of Usi...,https://doi.org/10.1001/amajethics.2019.180,2019-02-01,56,Ethical Considerations in Medical Research Par...


In [21]:
topics = []
# transform the topics into more general topics
for topic, title in zip(main_medicine_ai["topics"], main_medicine_ai["title"]):
    if "Language" in topic:
        topics.append("NLP and LLM")
    elif "language" in topic:
        topics.append("NLP and LLM")
    # LLMs are not used as a topic which is why the title is checked (for all the relevant papers GPT is used in the context of LLMs)
    elif "GPT" in title:
        topics.append("NLP and LLM")
    elif "Images" in topic:
        topics.append("Computer Vision")
    elif "Image" in topic:
        topics.append("Computer Vision")
    elif "Imaging" in topic:
        topics.append("Computer Vision")

    elif "Explainable" in topic:
        topics.append("Explainable")
    else:
        topics.append("Rest")

main_medicine_ai_edit = main_medicine_ai.copy()
main_medicine_ai_edit["topics"] = topics

main_medicine_ai_edit = (
    main_medicine_ai_edit.drop_duplicates().sort_values("topics").groupby("doi").first().reset_index()
)

Sorting and selecting the first ensures that the 'Rest' group is only used when none of the other topics are used in a paper. 

In [37]:
import plotly.express as px

# take the top 100 papers (each paper appears 3 times so we actually take the top 34 papers)
main_medicine_ai_edit = main_medicine_ai_edit.sort_values(by="citations", ascending=False).head(100)


color_map = {
    "Computer Vision": image_color[0],
    "NLP and LLM": nlp_color[0],
    "Explainable": explainable_color[0],
    "Rest": color_rest[0],
}

# create scatter plot
fig = px.scatter(
    data_frame=main_medicine_ai_edit,
    x="publication_date",
    y="citations",
    color="topics",
    symbol="topics",
    hover_data={"title": True},
    title="<b>Most Cited Papers</b> in Medicine with AI",
    color_discrete_map=color_map,
)


# Add anotations as legend replacement
fig.add_annotation(
    x="2019-09-13",
    y=5500,
    text="Explainable AI",
    ax=0,
    ay=0,
    font=dict(size=16, color=explainable_color[0], weight="bold"),
)

fig.add_annotation(
    x="2020-12-15", y=2500, text="Computer Vision", ax=0, ay=0, font=dict(size=16, color=image_color[0], weight="bold")
)
fig.add_annotation(
    x="2023-08-15", y=2000, text="NLP and LLMs", ax=0, ay=0, font=dict(size=16, color=nlp_color[0], weight="bold")
)

fig.add_annotation(
    x="2019-03-15", y=2500, text="Other", ax=0, ay=0, font=dict(size=16, color=color_rest[0], weight="bold")
)

# add caption
fig.add_annotation(
    xref="paper",
    yref="paper",
    xanchor="right",
    yanchor="top",
    x=1.0,
    y=1.0,
    ax=0,
    ay=0,
    align="left",
    text=f'The two <b>most cited papers</b> in Medicine with AI since 2019 are<br>related to <b><span style="color: {explainable_color[0]};">Explainable AI</span></b>. <b><span style="color: {nlp_color[0]};">Large Language Models</span></b> and NLP<br>seems to be an <b>emerging trend</b>, with papers published<br>in 2023 already having a high number of citations.',
)


fig.update_layout(
    template="simple_white",
    showlegend=False,
    xaxis_title="",
    yaxis_title="Citations",
    width=1000,
    height=600,
    margin=dict(l=120),
    yaxis=dict(range=[0, 6000]),
)
fig.write_image("citations_plot.png", scale=10)
fig.write_image("citations_plot.svg", scale=10)
fig.show()

The legend is removed and replaced with annotations that appear close to the actual data points, making the plot easier to understand. Again, the grid lines and background are removed to create a cohesive, uncluttered visualization. The main takeaway is explained in the caption, and the headline is kept simple to improve comprehension, as placing relevant information and the main takeaway in the title would make it too heavy. 